In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('..', 'TFIDF'))
sys.path.append(os.path.join('..', 'util'))
from timer import Timer
from sklearn import metrics

In [2]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [2]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:20000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_sm.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_sm.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000


In [22]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
# tfidf.calculate_tfidf_norms()
serial_cossims = tfidf.calculate_cossims()
serial_simhashes = tfidf.calculate_simhashes()
serial_distances = tfidf.calculate_distances()

Time for initialization: 0.0242218971252
Time for load_questions: 1.90734863281e-06
Time for load_indices: 0.0130228996277
Time for init_tfs: 0.927615880966
Time for create_tfs: 1.074021101
Time for create_idf: 1.49139118195
Time for calculate_tfidfs: 0.655948877335
Time for calculate_cossims: 32.8535630703
Time for calculate_simhashes: 86.8444750309
Time for calculate_distances: 35.4312767982
Time for calculate_cossims: 34.7172560692


In [23]:
serial_cossims.shape

(10000, 10000)

In [24]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 64)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances = np.asarray(tfidf_c.calculate_distances())

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 1.4066696167e-05
Time for load_questions: 0.168163061142
Time for load_indices: 0.0352330207825
Time for init_tfs: 0.671391010284
Time for create_tfs: 0.148532867432
Time for calculate_idf: 0.202357053757
Time for int_tfidfs: 0.612493991852
Time for calculate_tfidfs: 0.112776994705
Time for calculate_simhashes: 0.349174976349
Time for calculate distances: 0.853644132614


In [25]:
# Verification
print np.linalg.norm(cython_tfs - serial_tfs)
print np.linalg.norm(cython_idf - serial_idf)
print np.linalg.norm(cython_tfidfs - serial_tfidfs)
print np.linalg.norm(cython_simhashes - serial_simhashes)
print np.linalg.norm(cython_distances - serial_distances)

0.0
3.05905382673e-05
0.000547179457161
0.0
0.0


In [ ]:
#The following are from https://yesteapea.wordpress.com/2013/03/03/counting-the-number-of-set-bits-in-an-integer/
def numBits64(i):
    i = i - ((i >> np.uint64(1)) & np.uint64(0x5555555555555555))
    i = (i & np.uint64(0x3333333333333333)) + ((i >> np.uint64(2)) & np.uint64(0x3333333333333333))
    i = ((i + (i >> np.uint64(4))) & np.uint64(0x0F0F0F0F0F0F0F0F))
    return (i*(np.uint64(0x0101010101010101)))>>np.uint64(56)

def numBits32(i):
    i = i - ((i >> 1) & 0x55555555)
    i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
    i = ((i + (i >> 4)) & 0x0F0F0F0F)
    return (i*(0x01010101))>>24

In [ ]:
A = np.array([serial_simhashes] * len(serial_simhashes))
B = np.copy(A)
distances = numBits64(A ^ B.T)

In [26]:
from sklearn.cluster import DBSCAN

In [27]:
db = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances)

In [28]:
sum(map(lambda x: 1 if x >= 0 else 0, db.labels_))

338

In [68]:
set(db.labels_)

In [12]:
all_one_labels = []
counter = 0
for i in xrange(len(db.labels_)):
    if db.labels_[i] == -1:
        counter += 1
    elif db.labels_[i] == 4:
        all_one_labels.append(i)
print counter
print all_one_labels

1988
[]


In [ ]:
for i in all_one_labels: 
    print " ".join(question_texts[i])
    print ""

Now doing it for the 32BIT cython distances

In [69]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 32)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances32 = tfidf_c.calculate_distances()

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 1.21593475342e-05
Time for load_questions: 0.105110883713
Time for load_indices: 0.0522270202637
Time for init_tfs: 0.770179986954
Time for create_tfs: 0.169428825378
Time for calculate_idf: 0.1989839077
Time for int_tfidfs: 0.597656011581
Time for calculate_tfidfs: 0.11490893364
Time for calculate_simhashes: 0.235023021698
Time for calculate distances: 1.72152400017


In [70]:
db32 = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances32)

In [71]:
sum(map(lambda x: 1 if x >= 0 else 0, db32.labels_))

272

Cosine Similarity

In [65]:
dbCossim = DBSCAN(metric="precomputed", eps=0.9, min_samples=3).fit(serial_cossims)

In [66]:
sum(map(lambda x: 1 if x >= 0 else 0, dbCossim.labels_))

10000

In [67]:
set(dbCossim.labels_)

{0}

In [52]:
max(serial_cossims[0][1:])


0.42025060383200469

In [72]:
metrics.adjusted_rand_score(db.labels_, db32.labels_)

0.6208851443677339

In [35]:
metrics.adjusted_rand_score(dbCossim.labels_, db.labels_)

0.0

In [19]:
metrics.homogeneity_score(dbCossim.labels_, db.labels_)

1.0